In [1]:
from plum_chatbot.configs.settings import Settings
from qdrant_client import QdrantClient

settings = Settings()

client = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)

/tmp/ipykernel_10024/3374820657.py:6: UserWarning: Api key is used with an insecure connection.
  client = QdrantClient(url=settings.QDRANT_URL, api_key=settings.QDRANT_API_KEY)


In [2]:
client.info()

VersionInfo(title='qdrant - vector search engine', version='1.14.1', commit='530430fac2a3ca872504f276d2c91a5c91f43fa0')

In [3]:
from qdrant_client.models import Distance, VectorParams

client.create_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [4]:
from qdrant_client.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44], payload={"city": "Mumbai"}),
    ],
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [5]:
search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    with_payload=False,
    limit=3
).points

print(search_result)

[ScoredPoint(id=4, version=0, score=1.362, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=1.273, payload=None, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=1.208, payload=None, vector=None, shard_key=None, order_value=None)]


In [9]:
from qdrant_client.models import Filter, FieldCondition, MatchValue

search_result = client.query_points(
    collection_name="test_collection",
    query=[0.2, 0.1, 0.9, 0.7],
    query_filter=Filter(
        must=[FieldCondition(key="city", match=MatchValue(value="London"))]
    ),
    with_payload=True,
    limit=3,
).points

print(search_result)

[ScoredPoint(id=2, version=0, score=0.871, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]
